In [1]:
!git clone https://github.com/inttx/DLAM_SealedSurfaces.git
%cd DLAM_SealedSurfaces

Cloning into 'DLAM_SealedSurfaces'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 124 (delta 71), reused 77 (delta 35), pack-reused 0 (from 0)
Receiving objects: 100% (124/124), 163.94 KiB | 6.83 MiB/s, done.
Resolving deltas: 100% (71/71), done.
/home/janina/PycharmProjects/DLAM_SealedSurfaces/DLAM_SealedSurfaces


In [1]:
!pip install -r requirements.txt
%cd src


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
/home/janina/PycharmProjects/DLAM_SealedSurfaces/src


In [2]:
from settings import *
from dataset import PotsdamDataset, get_data_loaders
from models import get_trained_custom_resnet18, get_trained_deeplabv3_resnet101, get_trained_segformer_model
from evaluate import evaluate_segmentation

In [3]:
try:
    import os
    import shutil
    from google.colab import drive
    MOUNTPOINT = '/content/drive/'
    drive.mount(MOUNTPOINT)

    IMAGE_PATH = os.path.join(MOUNTPOINT, 'MyDrive', 'DLAM', '2_Ortho_RGB') # TODO adjust to your path
    LABEL_PATH = os.path.join(MOUNTPOINT, 'MyDrive', 'DLAM', '5_Labels_all') # TODO adjust to your path

    EVAL_PATH = MOUNTPOINT + 'MyDrive/DLAM/eval/'
except:
    IMAGE_PATH = '../data/2_Ortho_RGB' # TODO adjust to your path
    LABEL_PATH = '../data/5_Labels_all' # TODO adjust to your path
    EVAL_PATH = '../eval/'
os.makedirs(EVAL_PATH, exist_ok=True)

In [12]:
# Hyperparameters
patch_size = 250
stride = 250
batch_size = 8
num_classes = PotsdamDataset.get_num_classes(mode="single")
class_names = PotsdamDataset.CLASS_NAMES_SINGLE

6


# Train resnet18

In [5]:
dataset = PotsdamDataset(IMAGE_PATH, LABEL_PATH, patch_size=patch_size, stride=stride, device=DEVICE)
train_loader, val_loader, test_loader = get_data_loaders(dataset, dist=[0.8, 0.1, 0.1], batch_size=batch_size)

model = get_trained_custom_resnet18(model_path='../models/resnet18.pth', patch_size=patch_size, num_classes=num_classes, device=DEVICE)
evaluate_segmentation(model=model, data_loader=test_loader, device=DEVICE, model_type='ResNet18', class_names=class_names, patch_size=patch_size, eval_path=EVAL_PATH + 'resnet18.svg')

Evaluating: 100%|██████████| 274/274 [01:25<00:00,  3.20it/s]
/home/janina/PycharmProjects/DLAM_SealedSurfaces/src/evaluate.py:83: RuntimeWarning: invalid value encountered in divide
  per_class_iou = np.where(union > 0, intersection / union, 0.0)


Evaluation completed.
Pixel Accuracy: 0.6414
Mean IoU: 0.3597
Per-class IoU:
  Class 0: 0.5105
  Class 1: 0.5755
  Class 2: 0.4061
  Class 3: 0.3663
  Class 4: 0.2998
  Class 5: 0.0000


# Train DeepLabV3 resnet101 baseline

In [16]:
train_loader, val_loader, test_loader = get_data_loaders(IMAGE_PATH, LABEL_PATH,patch_size,
                     stride,
                     dist=[0.8, 0.1, 0.1],
                     batch_size=batch_size,
                     device=DEVICE,
                     mode="single",
                     pin_memory=False,
                     num_workers=0,
                     seed=42)

model = get_trained_deeplabv3_resnet101(model_path='../models/deeplab_stage3.pth', num_classes=num_classes, device=DEVICE)

evaluate_segmentation(model=model, data_loader=test_loader, device=DEVICE, model_type='DeepLabV3', class_names=class_names, patch_size=patch_size, eval_path=EVAL_PATH + 'deeplabv3_resnet101.svg')

Evaluating: 100%|██████████| 274/274 [08:52<00:00,  1.94s/it]


Evaluation completed.
Pixel Accuracy: 0.8661
Mean IoU: 0.7236
Per-class IoU:
  Class 0: 0.7934
  Class 1: 0.8893
  Class 2: 0.7493
  Class 3: 0.7040
  Class 4: 0.7006
  Class 5: 0.5048


# Train SegFormer

In [14]:
train_loader, val_loader, test_loader = get_data_loaders(IMAGE_PATH, LABEL_PATH,patch_size,
                     stride,
                     dist=[0.8, 0.1, 0.1],
                     batch_size=batch_size,
                     device=DEVICE,
                     mode="single",
                     pin_memory=False,
                     num_workers=0,
                     seed=42)

model = get_trained_segformer_model('../models/segformer_stage3.pth', num_classes=num_classes, device=DEVICE)

evaluate_segmentation(model=model, data_loader=test_loader, device=DEVICE, model_type='SegFormer', class_names=class_names, patch_size=patch_size, eval_path=EVAL_PATH + 'segformer.svg')

/home/janina/PycharmProjects/DLAM_SealedSurfaces/venv/lib/python3.12/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/home/janina/PycharmProjects/DLAM_SealedSurfaces/src/dataset.py:27: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
Building index: 100%|██████████| 24/24 [00:00<00:00, 202135.13it/s]
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([6]) in the model instantiated
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([6, 256, 1, 1]) in the model instantia

Evaluation completed.
Pixel Accuracy: 0.8644
Mean IoU: 0.7241
Per-class IoU:
  Class 0: 0.7878
  Class 1: 0.8768
  Class 2: 0.7368
  Class 3: 0.7079
  Class 4: 0.6925
  Class 5: 0.5426
